In [1]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import xgboost as xgb

%matplotlib inline

In [2]:
import os
rootpath = r"/home/bingo/桌面/sales_forecasting/steel_storage_throughput_prediction"
os.chdir(rootpath)

### 训练集整理函数
- selectData:加载原始数据，剔除无效数据
- loadData:加载数据整理为时间表

#### 清洗数据
- 预测任务
    - 1.按照两大类货品类型（冷卷、热卷），分别预测未来4个周钢铁的周入库量和周出库量（重量）；
    - 2.按照两大类货品类型（冷卷、热卷），分别预测未来i天的日入库量和日出库量（重量）。
- 只有产品名称是热卷和圆钢为热卷类型，其余均为冷卷类型
- 整理目标：
    - 仅筛选出包含冷卷和热卷两种货品的数据集，添加新列表示冷卷或热卷
    - 丢弃，数量=0而重量>0 重量=0而数量>0的数据

In [3]:
def get_item(dataset, target_col, item_list, new_col=None):
    df = pd.DataFrame(columns=dataset.columns)
    for item in item_list:
        temp = dataset[dataset[target_col] == item]
        df = pd.concat([df, temp], axis=0)
    if new_col:
        for key, item in new_col.items():
            df[key] = item
    return df

In [4]:
def selectData(inpath, extpath):
    # 读取原始数据
    df_train_in = pd.read_csv(inpath)
    df_train_ext = pd.read_csv(extpath)
    # 剔除无效数据
    print("df_train_in shape: ", df_train_in.shape)
    dropind = df_train_in[df_train_in["r2019"] == 0].index
    df_train_in = df_train_in.drop(dropind)
    dropind = df_train_in[df_train_in["r2018"] == 0].index
    df_train_in = df_train_in.drop(dropind)
    print("df_train_in shape: ", df_train_in.shape)

    print("df_train_ext shape: ", df_train_ext.shape)
    dropind = df_train_ext[df_train_ext["c2019"] == 0].index
    df_train_ext = df_train_ext.drop(dropind)
    dropind = df_train_ext[df_train_ext["c2018"] == 0].index
    df_train_ext = df_train_ext.drop(dropind)
    print("df_train_ext shape: ", df_train_ext.shape)

    # 设置种类
    cool_col = {"type": "冷卷", "type_id": 0}
    hot_col = {"type": "热卷", "type_id": 1}

    hot_in = ["热卷", "圆钢"]
    cool_in = set(df_train_in["r2017"].unique()) - set(hot_in)

    train_in_hot = get_item(df_train_in, "r2017", hot_in, hot_col)
    train_in_cool = get_item(df_train_in, "r2017", cool_in, cool_col)
    train_in_both = pd.concat([train_in_hot, train_in_cool], axis=0)
    del train_in_hot, train_in_cool

    hot_ext = ["热卷", "圆钢"]
    cool_ext = set(df_train_ext["c2017"].unique()) - set(hot_ext)

    train_ext_hot = get_item(df_train_ext, "c2017", hot_ext, hot_col)
    train_ext_cool = get_item(df_train_ext, "c2017", cool_ext, cool_col)
    train_ext_both = pd.concat([train_ext_hot, train_ext_cool], axis=0)
    del train_ext_cool, train_ext_hot

    # 出入库数据集样本数
    print("train_in samples: ", train_in_both.shape)
    print("train_ext samples: ", train_ext_both.shape)

    # 保存
    train_in_both.to_csv("data/train_in.csv", index=False)
    train_ext_both.to_csv("data/train_ext.csv", index=False)
    
    print("selecting data Done!")

#### 数据转换
- 整理目标
    - 转换为储户id-产品id-日期格式三层索引,重量为训练值，做为训练集格式
    - 无记录时间记出入库重量为0
    - 补齐中断的缺失时间，时间范围为(2014-02-24,2018-01-28)
    - 转换为储户id-产品id-日期格式三层索引,数量为训练值，做为训练集格式

In [5]:
def loadData(in_path, ext_path, endtime="2018-1-28"):
    starttime = date(2014, 2, 24)
    endtime = datetime.strptime(endtime,"%Y-%m-%d")
    # 进口重量训练集
    dtypes = {
        "r2014": str,
        "r2015": int,
        "r2019": float,
        "type_id": int,
    }
    use_cols = [0, 1, 5, 10]
    train_in = pd.read_csv(in_path, dtype=dtypes, usecols=use_cols)

    train_in["date"] = pd.to_datetime(
        train_in["r2014"].apply(lambda x: x[2:10]))

    # 选择数据列，更改列名
    select_col = ["date", "r2015", "r2019", "type_id"]
    new_names = ["date", "store_id", "sales", "type_id"]
    train_in = train_in[select_col]

    train_in.columns = new_names

    # 补全2014-2-24 2018-01-28日期内，缺失的时间及记录
    temp = train_in.groupby(["store_id", "type_id", "date"])[
        ["sales"]].sum().unstack(level=-1).fillna(0)
    temp.columns = temp.columns.get_level_values(1)

    # 2014-02-24为周一，2018-01-28为周日
    train_in_all = pd.DataFrame(temp, index=temp.index,
                                columns=pd.date_range(starttime, endtime))
    train_in_all.fillna(0, inplace=True)

    # 进口数量训练集
    dtypes = {
        "r2014": str,
        "r2015": int,
        "r2018": int,
        "type_id": int,
    }
    use_cols = [0, 1, 4, 10]
    train_in = pd.read_csv(in_path, dtype=dtypes, usecols=use_cols)

    train_in["date"] = pd.to_datetime(
        train_in["r2014"].apply(lambda x: x[2:10]))

    # 选择数据列，更改列名
    select_col = ["date", "r2015", "r2018", "type_id"]
    new_names = ["date", "store_id", "nums", "type_id"]
    train_in = train_in[select_col]

    train_in.columns = new_names

    # 补全2014-2-24 2018-01-28日期内，缺失的时间及记录
    temp = train_in.groupby(["store_id", "type_id", "date"])[
        ["nums"]].sum().unstack(level=-1).fillna(0)
    temp.columns = temp.columns.get_level_values(1)

    # 2014-02-24为周一，2018-01-28为周日
    train_in_all_nums = pd.DataFrame(temp, index=temp.index,
                                     columns=pd.date_range(starttime, endtime))
    train_in_all_nums.fillna(0, inplace=True)

    # 出口重量训练集
    dtypes = {
        "c2014": str,
        "c2015": int,
        "c2019": float,
        "type_id": int,
    }
    use_cols = [0, 1, 5, 10]
    train_ext = pd.read_csv(ext_path, dtype=dtypes, usecols=use_cols)

    train_ext["date"] = pd.to_datetime(train_ext["c2014"].
                                       apply(lambda x: "".join(list(filter(str.isdigit, x))[:8])))

    select_col = ["date", "c2015", "c2019", "type_id"]
    new_names = ["date", "store_id", "sales", "type_id"]

    train_ext = train_ext[select_col]
    train_ext.columns = new_names

    temp = train_ext.groupby(["store_id", "type_id", "date"])[
        ["sales"]].sum().unstack(level=-1).fillna(0)
    temp.columns = temp.columns.get_level_values(1)

    train_ext_all = pd.DataFrame(temp, index=temp.index,
                                 columns=pd.date_range(starttime, endtime))
    train_ext_all.fillna(0, inplace=True)

    # 出口数量集
    dtypes = {
        "c2014": str,
        "c2015": int,
        "c2018": int,
        "type_id": int,
    }
    use_cols = [0, 1, 4, 10]
    train_ext = pd.read_csv(ext_path, dtype=dtypes, usecols=use_cols)

    train_ext["date"] = pd.to_datetime(train_ext["c2014"].
                                       apply(lambda x: "".join(list(filter(str.isdigit, x))[:8])))

    select_col = ["date", "c2015", "c2018", "type_id"]
    new_names = ["date", "store_id", "nums", "type_id"]
    train_ext = train_ext[select_col]
    train_ext.columns = new_names

    temp = train_ext.groupby(["store_id", "type_id", "date"])[
        ["nums"]].sum().unstack(level=-1).fillna(0)
    temp.columns = temp.columns.get_level_values(1)

    train_ext_all_nums = pd.DataFrame(temp, index=temp.index,
                                      columns=pd.date_range(starttime, endtime))
    train_ext_all_nums.fillna(0, inplace=True)
    
    # 保存数据
    train_in_all.to_csv("data/train_in_all.csv")
    train_ext_all.to_csv("data/train_ext_all.csv")
    train_in_all_nums.to_csv("data/train_in_all_nums.csv")
    train_ext_all_nums.to_csv("data/train_ext_all_nums.csv")
    
    print("loading data Done!")
    return train_in_all, train_ext_all, train_in_all_nums, train_ext_all_nums

#### 使用

In [6]:
old_in = r"data_org/trainData_IN.csv"
old_ext = r"data_org/trainData_EXT.csv"
new_in = r"data_org/testData4_IN.csv"
new_ext = r"data_org/testData4_EXT.csv"
# 合并的原始数据集位置
inpath = r"data_org/train_IN.csv"
extpath = r"data_org/train_EXT.csv"

in1 = pd.read_csv(old_in)
ext1 = pd.read_csv(old_ext)
in2 = pd.read_csv(new_in)
ext2 = pd.read_csv(new_ext)

in3 = pd.concat([in1, in2], axis=0)
ext3 = pd.concat([ext1, ext2], axis=0)
in3.to_csv(inpath, index=False)
ext3.to_csv(extpath, index=False)
print("in1 shape: ",in1.shape)
print("in2 shape: ", in2.shape)
print("in3 shape: ", in3.shape)

in1 shape:  (332937, 9)
in2 shape:  (14713, 9)
in3 shape:  (347650, 9)


In [7]:
# 合并的原始数据集位置
inpath = r"data_org/train_IN.csv"
extpath = r"data_org/train_EXT.csv"
# 训练数据集位置
in_path = "data/train_in.csv"
ext_path = "data/train_ext.csv"
endtime = "2018-6-17"

selectData(inpath, extpath)
train_in_all, train_ext_all, train_in_all_nums, train_ext_all_nums = loadData(
    in_path, ext_path, endtime=endtime)
print("train_in_all shape: ", train_in_all.shape)

df_train_in shape:  (347650, 9)
df_train_in shape:  (346906, 9)
df_train_ext shape:  (425405, 9)
df_train_ext shape:  (425359, 9)
train_in samples:  (346906, 11)
train_ext samples:  (425359, 11)
selecting data Done!
loading data Done!
train_in_all shape:  (1386, 1575)


### 特征工程
- 滑窗选取时间窗，提取时间窗范围的统计量作为特征
- 有大量稀疏值，可以选择合并，可有效降低误差
- 数据存在异方差性，取log1p减少异方差，可降低误差
- 滑动平均平滑原始数据，可降低误差
- 周数据：截取时间段->/1000->ewm->merge->np.log1p->resample
    - 周数据预测使用累计方式计算，然后逐次相减
    - resample代表按1周、2周、3周、4周重采样
- 天数据：截取时间段->/1000->ewm->merge->np.log1p

In [8]:
# 合并含有大量0值的行
def merge_zeros_cols(df, merge_end=500, merge_step=50):
    freq = df.columns.freqstr
    notzeros = np.count_nonzero(df, axis=1)
    merge_list = np.arange(0,merge_end+1,merge_step)
    df = df.reset_index()
    df_ = df[notzeros>=merge_end].set_index("type_id")
    for i, value in enumerate(merge_list[:-1]):
        ind = (notzeros>=value)&(notzeros<merge_list[i+1])
        tmp = df.iloc[ind,:]
        tmp = tmp.groupby("type_id").sum(axis=0)
        tmp["store_id"] = merge_list[i+1]
        df_ = pd.concat([df_,tmp],axis=0)
    df_ = df_.reset_index().set_index(["store_id","type_id"])
    df_.columns = pd.DatetimeIndex(df_.columns,freq=freq)
    return df_

#### 周数据处理

In [9]:
span=28

df_train_in = pd.read_csv("data/train_in_all.csv").set_index(["store_id","type_id"])/1000
df_train_in.columns = pd.to_datetime(df_train_in.columns)
df_train_in.columns.name = "date"

df_train_ext = pd.read_csv("data/train_ext_all.csv").set_index(["store_id","type_id"])/1000
df_train_ext.columns = pd.to_datetime(df_train_ext.columns)
df_train_ext.columns.name = "date"

df_train_in_nums = pd.read_csv("data/train_in_all_nums.csv").set_index(["store_id","type_id"])/1000
df_train_in_nums.columns = pd.to_datetime(df_train_in_nums.columns)
df_train_in_nums.columns.name = "date"

df_train_ext_nums = pd.read_csv("data/train_ext_all_nums.csv").set_index(["store_id","type_id"])/1000
df_train_ext_nums.columns = pd.to_datetime(df_train_ext_nums.columns)
df_train_ext_nums.columns.name = "date"

df_train_in = df_train_in.ewm(span=span, axis=1).mean()
df_train_ext = df_train_ext.ewm(span=span, axis=1).mean()
df_train_in_nums = df_train_in_nums.ewm(span=span, axis=1).mean()
df_train_ext_nums = df_train_ext_nums.ewm(span=span, axis=1).mean()

print("df_train_in shape: ", df_train_in.shape)
print("df_train_ext_shape: ", df_train_ext.shape)
print("df_train_in_nums shape: ", df_train_in_nums.shape)
print("df_train_ext_nums shape: ", df_train_ext_nums.shape)

df_train_in shape:  (1386, 1575)
df_train_ext_shape:  (1383, 1575)
df_train_in_nums shape:  (1386, 1575)
df_train_ext_nums shape:  (1383, 1575)


In [10]:
merge_end = 600 
merge_step = 600 

tmp_in = merge_zeros_cols(df_train_in, merge_end, merge_step).transpose()
df_train_in_transpose = np.log1p(tmp_in)

tmp_ext = merge_zeros_cols(df_train_ext, merge_end, merge_step).transpose()
df_train_ext_transpose = np.log1p(tmp_ext)

tmp_in_nums = merge_zeros_cols(df_train_in_nums, merge_end, merge_step).transpose()
df_train_in_nums_transpose = tmp_in_nums  

tmp_ext_nums = merge_zeros_cols(df_train_ext_nums, merge_end, merge_step).transpose()
df_train_ext_nums_transpose = tmp_ext_nums  

# 按周重采样
df_train_in_W = df_train_in_transpose.resample(
    "W", closed="right", label="right").mean().transpose()
df_train_in_2W = df_train_in_transpose.resample(
    "2W", closed="right", label="right").mean().transpose()
df_train_in_3W = df_train_in_transpose.resample(
    "3W", closed="right", label="right").mean().transpose()
df_train_in_4W = df_train_in_transpose.resample(
    "4W", closed="right", label="right").mean().transpose()

df_train_ext_W = df_train_ext_transpose.resample(
    "W", closed="right", label="right").mean().transpose()
df_train_ext_2W = df_train_ext_transpose.resample(
    "2W", closed="right", label="right").mean().transpose()
df_train_ext_3W = df_train_ext_transpose.resample(
    "3W", closed="right", label="right").mean().transpose()
df_train_ext_4W = df_train_ext_transpose.resample(
    "4W", closed="right", label="right").mean().transpose()

# 数量重采样
df_train_in_nums_W = df_train_in_nums_transpose.resample(
    "W", closed="right", label="right").mean().transpose()
df_train_in_nums_2W = df_train_in_nums_transpose.resample(
    "2W", closed="right", label="right").mean().transpose()
df_train_in_nums_3W = df_train_in_nums_transpose.resample(
    "3W", closed="right", label="right").mean().transpose()
df_train_in_nums_4W = df_train_in_nums_transpose.resample(
    "4W", closed="right", label="right").mean().transpose()

df_train_ext_nums_W = df_train_ext_nums_transpose.resample(
    "W", closed="right", label="right").mean().transpose()
df_train_ext_nums_2W = df_train_ext_nums_transpose.resample(
    "2W", closed="right", label="right").mean().transpose()
df_train_ext_nums_3W = df_train_ext_nums_transpose.resample(
    "3W", closed="right", label="right").mean().transpose()
df_train_ext_nums_4W = df_train_ext_nums_transpose.resample(
    "4W", closed="right", label="right").mean().transpose()

print("df_train_in_W shape: ", df_train_in_W.shape)
print("df_train_ext_W shape: ", df_train_ext_W.shape)
print("df_train_in_nums_W shape: ", df_train_in_nums_W.shape)
print("df_train_ext_nums_W shape: ", df_train_ext_nums_W.shape)

df_train_in_W shape:  (650, 225)
df_train_ext_W shape:  (641, 225)
df_train_in_nums_W shape:  (650, 225)
df_train_ext_nums_W shape:  (641, 225)


#### 天数据处理

In [11]:
span=56

df_train_in = pd.read_csv("data/train_in_all.csv").set_index(["store_id","type_id"])/1000
df_train_in.columns = pd.to_datetime(df_train_in.columns)
df_train_in.columns.name = "date"

df_train_ext = pd.read_csv("data/train_ext_all.csv").set_index(["store_id","type_id"])/1000
df_train_ext.columns = pd.to_datetime(df_train_ext.columns)
df_train_ext.columns.name = "date"

df_train_in_nums = pd.read_csv("data/train_in_all_nums.csv").set_index(["store_id","type_id"])/1000
df_train_in_nums.columns = pd.to_datetime(df_train_in_nums.columns)
df_train_in_nums.columns.name = "date"

df_train_ext_nums = pd.read_csv("data/train_ext_all_nums.csv").set_index(["store_id","type_id"])/1000
df_train_ext_nums.columns = pd.to_datetime(df_train_ext_nums.columns)
df_train_ext_nums.columns.name = "date"

df_train_in = df_train_in.ewm(span=span, axis=1).mean()
df_train_ext = df_train_ext.ewm(span=span, axis=1).mean()
df_train_in_nums = df_train_in_nums.ewm(span=span, axis=1).mean()
df_train_ext_nums = df_train_ext_nums.ewm(span=span, axis=1).mean()

print("df_train_in shape: ", df_train_in.shape)
print("df_train_ext_shape: ", df_train_ext.shape)
print("df_train_in_nums shape: ", df_train_in_nums.shape)
print("df_train_ext_nums shape: ", df_train_ext_nums.shape)

df_train_in shape:  (1386, 1575)
df_train_ext_shape:  (1383, 1575)
df_train_in_nums shape:  (1386, 1575)
df_train_ext_nums shape:  (1383, 1575)


In [12]:
merge_end = 200 
merge_step = 200 

df_train_in = merge_zeros_cols(df_train_in, merge_end, merge_step)
df_train_ext = merge_zeros_cols(df_train_ext, merge_end, merge_step)

# 再直接取对数
df_train_in = np.log1p(df_train_in)
df_train_ext = np.log1p(df_train_ext)

# 数量处理
df_train_in_nums = merge_zeros_cols(df_train_in_nums, merge_end, merge_step)
df_train_ext_nums = merge_zeros_cols(df_train_ext_nums, merge_end, merge_step)

print("df_train_in shape: ", df_train_in.shape)
print("df_train_ext_shape: ", df_train_ext.shape)

df_train_in shape:  (1120, 1575)
df_train_ext_shape:  (1118, 1575)


#### 特征子集抽取

In [13]:
def prepare_dataset_byDay(df, dt, dnums=None, addType=False, is_train=True):
    x = {}
    n = 11
    times = [3,7,14,21,35,56] 
    for i in times:
        tmp = get_span(df, dt, i-1, i)
        x["sales_%s_sum"%(i)] = tmp.sum(axis=1).values
        x["sales_%s_mean"%(i)] = tmp.mean(axis=1).values
        x["sales_%s_sum_decay"%(i)] = \
        (tmp*np.power(0.999,np.arange(i)[::-1])).sum(axis=1).values
        x["sales_%s_mean_decay"%(i)] = \
        (tmp*np.power(0.999,np.arange(i)[::-1])).mean(axis=1).values
        
        tmp = get_span(dnums, dt, i-1, i)
        x["nums_%s_sum_decay"%(i)] = \
        (tmp*np.power(0.9,np.arange(i)[::-1])).sum(axis=1).values
        x["nums_%s_mean_decay"%(i)] = \
        (tmp*np.power(0.9,np.arange(i)[::-1])).mean(axis=1).values

    for i in times:
        tmp = get_span(df, dt, i-1, i)
        x["sales_%s_count"%(i)] = np.count_nonzero(tmp.values, axis=1) 
        x["no_sales_%s_count"%(i)] = i - x["sales_%s_count"%(i)]
        
    for i in times:
        tmp = get_span(df, dt, i-1, i)
        x["diff_%s_mean"%(i)] = tmp.diff(axis=1).mean(axis=1).values
        x["std_%s"%(i)] = tmp.std(axis=1).values
        
        tmp = get_span(dnums, dt, i-1, i)
        x["nums_diff_%s_mean"%(i)] = tmp.diff(axis=1).mean(axis=1).values
        x["nums_std_%s"%(i)] = tmp.std(axis=1).values
        
    for i in times:
        tmp = get_span(df, dt+timedelta(days=-14), i-1, i)
        x['diff_%s_mean_2' %(i)] = tmp.diff(axis=1).mean(axis=1).values
        x['mean_%s_decay_2' %(i)] = \
        (tmp * np.power(0.9, np.arange(i)[::-1])).mean(axis=1).values #sum
        
        tmp = get_span(dnums, dt+timedelta(days=-14), i-1, i)
        x["nums_diff_%s_mean_2"%(i)] = tmp.diff(axis=1).mean(axis=1).values
        x['nums_mean_%s_decay_2' %(i)] = \
        (tmp * np.power(0.9, np.arange(i)[::-1])).mean(axis=1).values #sum        
    
    for i in times:
        tmp = get_span(df, dt, i-1, i)
        x['has_sales_days_in_last_%s' %(i)] = (tmp > 0).sum(axis=1).values
        x["last_sales_days_in_last_%s"%(i)] =\
        i - ((tmp>0)*np.arange(i)).max(axis=1).values
        x["first_sales_days_in_last_%s"%(i)] =\
        ((tmp>0)*np.arange(i,0,-1)).max(axis=1).values
        
    for i in np.arange(7):                  
        x["nums_mean_4_dow%s"%(i)] = get_span(dnums, dt, 28-i, 4, freq="7D").mean(axis=1).values
        x["nums_mean_8_dow%s"%(i)] = get_span(dnums, dt, 56-i, 8, freq="7D").mean(axis=1).values
        x["nums_mean_12_dow%s"%(i)] = get_span(dnums, dt, 84-i, 12, freq="7D").mean(axis=1).values
        
    for i in np.arange(0,31):
        x["day_%s"%(i)] = get_span(df, dt, i, 1).values.ravel()
    
    if addType:
        x = pd.DataFrame(x)
        x["type"] = df.reset_index()["type_id"]
    else:
        x = pd.DataFrame(x)
    
    if is_train:
        y = df[pd.date_range(dt+timedelta(1),periods=7)].values
        return x,y
    return x

In [14]:
def get_span(df, dt, minus, periods, freq="D"):
    return df[pd.date_range(dt-timedelta(days=int(minus)), periods=periods, freq=freq)]

n = 5
t2018 = date(2018, 1, 14)+timedelta(28*n)
num_days = 20*5

print("Preparing dataset...")

x_in, y_in = [], []
x_ext, y_ext = [], []
addType = True

for i in range(num_days):
    delta = timedelta(days=i)

    x_tmp_in, y_tmp_in = prepare_dataset_byDay(
        df_train_in, t2018-delta, df_train_in_nums, addType=addType)
    x_tmp_ext, y_tmp_ext = prepare_dataset_byDay(
        df_train_ext, t2018-delta, df_train_ext_nums, addType=addType)

    x_in.append(x_tmp_in)
    y_in.append(y_tmp_in)
    x_ext.append(x_tmp_ext)
    y_ext.append(y_tmp_ext)

x_train_in = pd.concat(x_in, axis=0)
y_train_in = np.concatenate(y_in, axis=0)
x_train_ext = pd.concat(x_ext, axis=0)
y_train_ext = np.concatenate(y_ext, axis=0)

print("x_train_in : ", x_train_in.shape)
print("y_train_in : ", y_train_in.shape)
print("x_train_ext : ", x_train_ext.shape)
print("y_train_ext : ", y_train_ext.shape)

Preparing dataset...
x_train_in :  (112000, 167)
y_train_in :  (112000, 7)
x_train_ext :  (111800, 167)
y_train_ext :  (111800, 7)


In [15]:
# 验证集 2018-1-21
valdate = date(2018, 1, 21) + timedelta(28*n)
x_val_in, y_val_in = prepare_dataset_byDay(
    df_train_in,  valdate, df_train_in_nums, addType=addType)
x_val_ext, y_val_ext = prepare_dataset_byDay(
    df_train_ext,  valdate, df_train_ext_nums, addType=addType)
print("x_val_in shape: ", x_val_in.shape)

# 测试集 2018-1-28
# testdate = date(2018,1,28)
testdate = date(2018, 1, 28) + timedelta(28*n)
x_test_in = prepare_dataset_byDay(
    df_train_in, testdate, df_train_in_nums,  is_train=False, addType=addType)
x_test_ext = prepare_dataset_byDay(
    df_train_ext, testdate, df_train_ext_nums, is_train=False, addType=addType)
print("x_test_in shape: ", x_test_in.shape)

x_val_in shape:  (1120, 167)
x_test_in shape:  (1120, 167)


In [16]:
def prepare_dataset_byWeek(df, dt, dnums=None, step=7, addType=False, is_train=True):
    x = {}
    n = 11
    times = [1, 3, 5, 7, 10]
    freq = df.columns.freqstr
    if not freq:
        if step==14: freq = "2W-SUN"
        elif step==21: freq="3W-SUN"
        else: freq="4W-SUN"
            
    for i in times:  
        tmp = get_span(df, dt, i*step, i, freq=freq)
        x["sales_%s_sum"%(i)] = tmp.sum(axis=1).values
        x["sales_%s_mean"%(i)] = tmp.mean(axis=1).values
        x["sales_%s_sum_decay" % (i)] = \
            (tmp*np.power(0.999, np.arange(i)[::-1])).sum(axis=1).values
        x["sales_%s_mean_decay" % (i)] = \
            (tmp*np.power(0.999, np.arange(i)[::-1])).mean(axis=1).values

        tmp = get_span(dnums, dt, i*step, i, freq=freq)
        x["nums_%s_sum_decay" % (i)] = \
            (tmp*np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        x["nums_%s_mean_decay" % (i)] = \
            (tmp*np.power(0.9, np.arange(i)[::-1])).mean(axis=1).values

    for i in times:  
        tmp = get_span(df, dt, i*step, i, freq=freq)
        x["sales_%s_count" % (i)] = np.count_nonzero(tmp.values, axis=1)
        x["no_sales_%s_count" % (i)] = i - x["sales_%s_count" % (i)]

    for i in times:  
        tmp = get_span(df, dt, i*step, i, freq=freq)
        x["diff_%s_mean" % (i)] = tmp.diff(axis=1).mean(axis=1).values
        x["std_%s" % (i)] = tmp.std(axis=1).values

        tmp = get_span(dnums, dt, i*step, i, freq=freq)
        x["nums_diff_%s_mean" % (i)] = tmp.diff(axis=1).mean(axis=1).values
        x["nums_std_%s" % (i)] = tmp.std(axis=1).values

    for i in times:  
        n = step if step % 14 or 14 % step else 14
        tmp = get_span(df, dt+timedelta(days=-n), i*step, i, freq=freq)
        x['diff_%s_mean_2' % (i)] = tmp.diff(axis=1).mean(axis=1).values
        x['mean_%s_decay_2' % (i)] = \
            (tmp * np.power(0.9, np.arange(i)
                            [::-1])).mean(axis=1).values  # sum

        tmp = get_span(dnums, dt+timedelta(days=-n), i*step, i, freq=freq)
        x["nums_diff_%s_mean_2" % (i)] = tmp.diff(axis=1).mean(axis=1).values
        x['nums_mean_%s_decay_2' % (i)] = \
            (tmp * np.power(0.9, np.arange(i)
                            [::-1])).mean(axis=1).values  

    for i in times:  
        tmp = get_span(df, dt, i*step, i, freq=freq)
        x['has_sales_days_in_last_%s' % (i)] = (tmp > 0).sum(axis=1).values
        x["last_sales_days_in_last_%s" % (i)] =\
            i - ((tmp > 0)*np.arange(i)).max(axis=1).values
        x["first_sales_days_in_last_%s" % (i)] =\
            ((tmp > 0)*np.arange(i, 0, -1)).max(axis=1).values

    for i in np.arange(1, 8):  
        x["day_%s" % (i)] = get_span(
            df, dt, i*step, 1, freq=freq).values.ravel()

    if addType:
        x = pd.DataFrame(x)
        x["type"] = df.reset_index()["type_id"]
    else:
        x = pd.DataFrame(x)

    if is_train:
        y = df[pd.date_range(dt, periods=1, freq=freq)].values
        return x, y
    return x

In [17]:
def getDatas(df_in, df_ext, df_in_nums=None, df_ext_nums=None, dt="2018-01-21", step=7, num_days=int(15), is_train=True, addType=True):
    print("Preparing dataset...step", step)
    if isinstance(dt, str):
        dt = datetime.strptime(dt, "%Y-%m-%d")
    num_days = num_days
    x_in, y_in = [], []
    x_ext, y_ext = [], []
    for i in range(num_days):
        delta = timedelta(days=i*step)
        if is_train:
            x_tmp_in, y_tmp_in = prepare_dataset_byWeek(
                df_in, dt-delta, df_in_nums, step=step, addType=addType, is_train=is_train)
            x_tmp_ext, y_tmp_ext = prepare_dataset_byWeek(
                df_ext, dt-delta, df_ext_nums, step=step, addType=addType, is_train=is_train)
        else:
            x_tmp_in = prepare_dataset_byWeek(
                df_in, dt-delta, df_in_nums, step=step, addType=addType, is_train=is_train)
            x_tmp_ext = prepare_dataset_byWeek(
                df_ext, dt-delta, df_ext_nums, step=step, addType=addType, is_train=is_train)
            y_tmp_in = []
            y_tmp_ext = []

        x_in.append(x_tmp_in)
        y_in.append(y_tmp_in)
        x_ext.append(x_tmp_ext)
        y_ext.append(y_tmp_ext)

    x_train_in = pd.concat(x_in, axis=0)
    y_train_in = np.concatenate(y_in, axis=0)
    x_train_ext = pd.concat(x_ext, axis=0)
    y_train_ext = np.concatenate(y_ext, axis=0)
    return x_train_in, y_train_in, x_train_ext, y_train_ext

In [18]:
def get_span(df, dt, minus, periods, freq="D"):
    return df[pd.date_range(dt-timedelta(days=int(minus)), periods=periods, freq=freq)]

i = -2
train_date = [
    df_train_in_W.columns[i],
    df_train_in_2W.columns[i],
    df_train_in_3W.columns[i],
    df_train_in_4W.columns[i]
]

# 训练集
x_train_in_W, y_train_in_W, x_train_ext_W, y_train_ext_W = getDatas(
    df_train_in_W, df_train_ext_W, df_train_in_nums_W, df_train_ext_nums_W, train_date[0], 7)

x_train_in_2W, y_train_in_2W, x_train_ext_2W, y_train_ext_2W = getDatas(
    df_train_in_2W, df_train_ext_2W, df_train_in_nums_2W, df_train_ext_nums_2W, train_date[1], 14)

x_train_in_3W, y_train_in_3W, x_train_ext_3W, y_train_ext_3W = getDatas(
    df_train_in_3W, df_train_ext_3W, df_train_in_nums_3W, df_train_ext_nums_3W, train_date[2], 21)

x_train_in_4W, y_train_in_4W, x_train_ext_4W, y_train_ext_4W = getDatas(
    df_train_in_4W, df_train_ext_4W, df_train_in_nums_4W, df_train_ext_nums_4W, train_date[3], 28)
print("x_train_in_W shape: ", x_train_in_W.shape)
print("x_train_in_2W shape: ", x_train_in_2W.shape)

# 验证集
i=-1
val_date = [
    df_train_in_W.columns[i],
    df_train_in_2W.columns[i],
    df_train_in_3W.columns[i],
    df_train_in_4W.columns[i]
]  
x_val_in_W, y_val_in_W, x_val_ext_W, y_val_ext_W = getDatas(
    df_train_in_W, df_train_ext_W, df_train_in_nums_W, df_train_ext_nums_W, val_date[0], 7, 1)

x_val_in_2W, y_val_in_2W, x_val_ext_2W, y_val_ext_2W = getDatas(
    df_train_in_2W, df_train_ext_2W,  df_train_in_nums_2W, df_train_ext_nums_2W, val_date[1], 14, 1)

x_val_in_3W, y_val_in_3W, x_val_ext_3W, y_val_ext_3W = getDatas(
    df_train_in_3W, df_train_ext_3W, df_train_in_nums_3W, df_train_ext_nums_3W, val_date[2], 21, 1)

x_val_in_4W, y_val_in_4W, x_val_ext_4W, y_val_ext_4W = getDatas(
    df_train_in_4W, df_train_ext_4W, df_train_in_nums_4W, df_train_ext_nums_4W, val_date[3], 28, 1)

print("x_val_in_W shape: ", x_val_in_W.shape)
print("x_val_in_2W shape: ", x_val_in_2W.shape)
print("x_val_ext_2W shape: ", y_val_ext_2W.shape)

# 测试集
test_date = []
steps = [7,14,21,28]
for i in range(4):
    test_date.append(val_date[i] + timedelta(steps[i]))

x_test_in_W, y_test_in_W, x_test_ext_W, y_test_ext_W = getDatas(
    df_train_in_W, df_train_ext_W, df_train_in_nums_W, df_train_ext_nums_W, test_date[0], 7, 1, False)

x_test_in_2W, y_test_in_2W, x_test_ext_2W, y_test_ext_2W = getDatas(
    df_train_in_2W, df_train_ext_2W, df_train_in_nums_2W, df_train_ext_nums_2W, test_date[1], 14, 1, False)

x_test_in_3W, y_test_in_3W, x_test_ext_3W, y_test_ext_3W = getDatas(
    df_train_in_3W, df_train_ext_3W, df_train_in_nums_3W, df_train_ext_nums_3W, test_date[2], 21, 1, False)

x_test_in_4W, y_test_in_4W, x_test_ext_4W, y_test_ext_4W = getDatas(
    df_train_in_4W, df_train_ext_4W, df_train_in_nums_4W, df_train_ext_nums_4W, test_date[3], 28, 1, False)
print("x_test_in_W shape: ", x_test_in_W.shape)
print("x_test_in_2W shape: ", x_test_ext_2W.shape)

Preparing dataset...step 7
Preparing dataset...step 14
Preparing dataset...step 21
Preparing dataset...step 28
x_train_in_W shape:  (9750, 103)
x_train_in_2W shape:  (9750, 103)
Preparing dataset...step 7
Preparing dataset...step 14
Preparing dataset...step 21
Preparing dataset...step 28
x_val_in_W shape:  (650, 103)
x_val_in_2W shape:  (650, 103)
x_val_ext_2W shape:  (641, 1)
Preparing dataset...step 7
Preparing dataset...step 14
Preparing dataset...step 21
Preparing dataset...step 28
x_test_in_W shape:  (650, 103)
x_test_in_2W shape:  (641, 103)


In [19]:
x_train_in_Wall = [x_train_in_W, x_train_in_2W, x_train_in_3W, x_train_in_4W]
y_train_in_Wall = np.concatenate(
    [y_train_in_W, y_train_in_2W, y_train_in_3W, y_train_in_4W], axis=1)
x_train_ext_Wall = [x_train_ext_W, x_train_ext_2W,
                    x_train_ext_3W, x_train_ext_4W]
y_train_ext_Wall = np.concatenate(
    [y_train_ext_W, y_train_ext_2W, y_train_ext_3W, y_train_ext_4W], axis=1)

x_val_in_Wall = [x_val_in_W, x_val_in_2W, x_val_in_3W, x_val_in_4W]
y_val_in_Wall = np.concatenate(
    [y_val_in_W, y_val_in_2W, y_val_in_3W, y_val_in_4W], axis=1)
x_val_ext_Wall = [x_val_ext_W, x_val_ext_2W, x_val_ext_3W, x_val_ext_4W]
y_val_ext_Wall = np.concatenate(
    [y_val_ext_W, y_val_ext_2W, y_val_ext_3W, y_val_ext_4W], axis=1)

x_test_in_Wall = [x_test_in_W, x_test_in_2W, x_test_in_3W, x_test_in_4W]
x_test_ext_Wall = [x_test_ext_W, x_test_ext_2W, x_test_ext_3W, x_test_ext_4W]

### 算法建模
- 天粒度预测
- 周粒度预测

#### 细粒度-1天1个模型

In [20]:
params1 = {
    "booster": "gblinear",
    "eta": 0.005,
    "alpha": 0,
    "lambda": 1,
    "nthread": 4,
    "objective": "reg:linear",
    "eval_metric": "rmse",
    "silent": 1,
}

In [21]:
# 一天一个模型
boosting_round = [100,200,400]
MAX_ROUNDS = np.sum(boosting_round)
etas = [0.01, 0.001, 0.0001]
eta_ = np.ones(MAX_ROUNDS)
eta_list = [np.tile(eta, rounds) for rounds, eta in zip(boosting_round, etas)]
eta_list = np.concatenate(eta_list, axis=0)

if not os.path.exists(r"./model/model1"):
    os.mkdir(r"./model/model1")
        
for i in range(7):
    print("="*50)
    print("step %d"%(i+1))
    print(datetime.now())
    print("="*50)
    dtrain = xgb.DMatrix(
        data=np.concatenate([x_train_in.values,x_train_ext.values],axis=0),
        label=np.concatenate([y_train_in[:,i], y_train_ext[:,i]], axis=0)
    )
    
    dval = xgb.DMatrix(
        data=np.concatenate([x_val_in.values,x_val_ext.values], axis=0),
        label=np.concatenate([y_val_in[:,i], y_val_ext[:,i]], axis=0)
    )
    
    bst = xgb.train(
        params1,
        dtrain,
        num_boost_round=MAX_ROUNDS,
        evals=[(dtrain,"dtrain"),(dval,"dval")],
        early_stopping_rounds=300,
        verbose_eval=500,
        callbacks=[xgb.callback.reset_learning_rate(list(eta_list))]
    )

    bst.save_model(r"./model/model1/bst_byday"+str(i+1)+".model")

step 1
2018-12-29 22:28:55.591979
[0]	dtrain-rmse:0.396163	dval-rmse:0.395486
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[500]	dtrain-rmse:0.010502	dval-rmse:0.008498
[699]	dtrain-rmse:0.010498	dval-rmse:0.008486
step 2
2018-12-29 22:29:37.379147
[0]	dtrain-rmse:0.39615	dval-rmse:0.395368
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[500]	dtrain-rmse:0.01092	dval-rmse:0.009125
[699]	dtrain-rmse:0.010916	dval-rmse:0.009114
step 3
2018-12-29 22:30:18.852348
[0]	dtrain-rmse:0.396139	dval-rmse:0.395372
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[500]	dtrain-rmse:0.011335	dval-rmse:0.009063
[699]	dtrain-rmse:0.011331	dval-rmse:0.009052
step 4
2018-12-29 22:31:00.272398
[0]	dtrain-rmse:0.396129	d

#### 粗粒度-4个累积模型

In [22]:
print("Training and Predicting models...")
params_w4 = {
    "booster":"gblinear",
    "eta":0.005,
    "alpha":0,
    "lambda":1,
    "nthread":4,
    "objective":"reg:linear",
    "eval_metric":"rmse",
    "silent":1
}

Training and Predicting models...


In [23]:
boosting_round = [100,200, 400]
MAX_ROUNDS = np.sum(boosting_round)
etas = [0.01, 0.001, 0.0001] 
eta_ = np.ones(MAX_ROUNDS)
eta_list = [np.tile(eta, rounds) for rounds, eta in zip(boosting_round, etas)]
eta_list = np.concatenate(eta_list, axis=0)

if not os.path.exists(r"./model/model1"):
        os.mkdir(r"./model/model1")
        
for i in range(4):
    print("="*50)
    print("step %d"%(i+1))
    print(datetime.now())
    print("="*50)
    dtrain_w4 = xgb.DMatrix(
        data=np.concatenate([x_train_in_Wall[i].values, x_train_ext_Wall[i].values],axis=0),
        label=np.concatenate([y_train_in_Wall[:,i], y_train_ext_Wall[:,i]], axis=0)
    )
    
    dval_w4 = xgb.DMatrix(
        data=np.concatenate([x_val_in_Wall[i].values,x_val_ext_Wall[i].values], axis=0),
        label=np.concatenate([y_val_in_Wall[:,i], y_val_ext_Wall[:,i]], axis=0)
    )
        
    bst_w4 = xgb.train(
        params_w4,
        dtrain_w4,
        num_boost_round=MAX_ROUNDS,
        evals=[(dtrain_w4,"dtrain"),(dval_w4,"dval")],
        early_stopping_rounds=300,
        verbose_eval=1000,
        callbacks=[xgb.callback.reset_learning_rate(list(eta_list))]
    )
    
    bst_w4.save_model(r"./model/model1/bst_byweek"+str(i+1)+".model")

step 1
2018-12-29 22:33:46.522929
[0]	dtrain-rmse:0.416029	dval-rmse:0.41575
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[699]	dtrain-rmse:0.0353	dval-rmse:0.0325
step 2
2018-12-29 22:33:53.468409
[0]	dtrain-rmse:0.416229	dval-rmse:0.415548
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[699]	dtrain-rmse:0.039435	dval-rmse:0.044357
step 3
2018-12-29 22:33:59.051657
[0]	dtrain-rmse:0.415886	dval-rmse:0.415353
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't improved in 300 rounds.
[699]	dtrain-rmse:0.040131	dval-rmse:0.051377
step 4
2018-12-29 22:34:03.551629
[0]	dtrain-rmse:0.415601	dval-rmse:0.41538
Multiple eval metrics have been passed: 'dval-rmse' will be used for early stopping.

Will train until dval-rmse hasn't impr

### 预测提交

#### 函数

In [24]:
def prediction(test, path="./model/model1",mode="wd"):
    if mode=="wd":
        week_ext, week_in, day_ext, day_in = [], [], [], []
        results = [week_ext, week_in, day_ext, day_in]
        bases = ["bst_byweek%s.model","bst_byweek%s.model","bst_byday%s.model","bst_byday%s.model"]
        steps = [0,0,7,7]
        for step, res, base, data in zip(steps, results, bases, test):
            dtest = xgb.DMatrix(data.values)
            for i in range(step):
                model = os.path.join(path, base%(i+1))
                print(model)
                bst = xgb.Booster()
                bst.load_model(model)
                res.append(bst.predict(dtest))
    else:
        W, W2, W3, W4 = [], [], [], []
        results = [W, W2, W3, W4]
        bases = ["bst_byweek%s.model"]*4
        steps = range(4)
        for step, res, base, data in zip(steps, results, bases, test):
            dtest = xgb.DMatrix(data.values)
            model = os.path.join(path, base%(step+1))
            print(model)
            bst = xgb.Booster()
            bst.load_model(model)
            res.append(bst.predict(dtest))
    return results

In [25]:
def getLastW(data):
    d1 = np.array(data).squeeze().transpose()
    d2 = np.expm1(d1)*np.array([7,14,21,28]) 
    d3 = np.diff(d2, axis=1)
    d2[:,1:]= d3
    d2 = np.clip(d2,0,None)
    res = np.log1p(d2).transpose()
    return res

In [26]:
def makeSubmission(preds, inds, steps=[4,4,7,7], filename="./submission/result.csv"):
    stack = []
    for ind,pred,step in zip(inds, preds, steps):
        pred = np.array(pred).transpose()
        tmp = pd.DataFrame(np.expm1(pred),index=ind) 
        tmp = tmp.reset_index().drop("store_id",axis=1)
        tmp = tmp.groupby("type_id").sum().values[::-1]
        tmp = tmp.reshape(2*step)
        stack.append(tmp)
    stack_pred = np.concatenate(stack, axis=0)
    stack_pred[:16] = stack_pred[:16]*1000
    stack_pred[16:] = stack_pred[16:]*1000
    result = pd.read_csv("./submission/result_org.csv")
    result["VALUE"] = stack_pred
    result.to_csv(filename,float_format="%.4f",index=False)
    print("done!")
    return stack_pred

#### 使用

In [27]:
x_test_in_Wall = [x_test_in_W, x_test_in_2W, x_test_in_3W, x_test_in_4W]
x_test_ext_Wall = [x_test_ext_W, x_test_ext_2W, x_test_ext_3W, x_test_ext_4W]
test = [x_test_ext_W, x_test_in_W, x_test_ext, x_test_in]

mpath = './model/model1'
res_ext_W = prediction(x_test_ext_Wall,path=mpath,mode="w")
res_in_W = prediction(x_test_in_Wall,path=mpath, mode="w")
res = prediction(test,path=mpath)

# 求各周
res1 = getLastW(res_ext_W)
res2 = getLastW(res_in_W)
results = [res1,res2,res[2],res[3]]

inds = [df_train_ext_W.index, df_train_in_W.index, df_train_ext.index, df_train_in.index]

# 取exp
pred1 = makeSubmission(results, inds)

./model/model1/bst_byweek1.model
./model/model1/bst_byweek2.model
./model/model1/bst_byweek3.model
./model/model1/bst_byweek4.model
./model/model1/bst_byweek1.model
./model/model1/bst_byweek2.model
./model/model1/bst_byweek3.model
./model/model1/bst_byweek4.model
./model/model1/bst_byday1.model
./model/model1/bst_byday2.model
./model/model1/bst_byday3.model
./model/model1/bst_byday4.model
./model/model1/bst_byday5.model
./model/model1/bst_byday6.model
./model/model1/bst_byday7.model
./model/model1/bst_byday1.model
./model/model1/bst_byday2.model
./model/model1/bst_byday3.model
./model/model1/bst_byday4.model
./model/model1/bst_byday5.model
./model/model1/bst_byday6.model
./model/model1/bst_byday7.model
done!
